In [1]:
from IPython.display import Image
from IPython.core.display import HTML

As we know, multiplying two matrices $A(m \times n)$ and $B(n \times p)$ results in matrix $AB(m \times p)$.
- Complexity of computing AB is $O(mnp)$

Matrix multiplication is associative:

$ABC = (AB)C = A(BC)$

Order gives the same result but, affects the complexity of computing. 

`Example`: Suppose we have matrices $A(1,100)$, $B(100,1)$ and $C(1,100)$. 
- Computing $BC(100, 100)$ will be $100 \times 1 \times 100$ and computing $A(BC)$ will then be $1 \times 100 \times 100$. Totalling 20000 steps. 

On the other hand
- Computing $AB(1, 1)$ will be $1 \times 100 \times 1$ and computing $(AB)C$ will then be $1 \times 1 \times 100$. Totalling 200 steps.


## Optimal way to multiply n such matrices, choose $k$

That is find bracketing structure, whether $(AB)C$ or $A(BC)$ for three matrices. Or for four matrices with 3 multiplications:

![Representation as a tree](./imgs/matMul-2025-01-09-2201.png)

 For n such matrices we need to find k such that $(M_1 \times M_2 \times M_3 \times \cdots \times M_k) \times (M_{k+1} \times M_{k+2} \times M_{k+3} \times \cdots \times M_n)$ for some $1 \leq k < n$. FIrst factor will have dim $(r_1,c_k)$ and other will have $(r_{k+1},c_n)$ with the cost $O(r_1c_kc_n)$.

### Subproblems
- Final step $(M_1 \times M_2 \times M_3 \times \cdots \times M_k) \times (M_{k+1} \times M_{k+2} \times M_{k+3} \times \cdots \times M_n)$

- Subproblem are $(M_1 \times M_2 \times M_3 \times \cdots \times M_k)$ and $(M_{k+1} \times M_{k+2} \times M_{k+3} \times \cdots \times M_n)$

- Total cost = Cost of Subproblem 1 + Cost of subproblem 2 + Cost of final problem which is $(r_1c_kc_n)$

- $n-1$ choices of $k$. Yet again we have $j$ for each subproblem $1 \cdots k$. So to `generalize`:

$Cost(M_i \times M_{i+1} \times  \cdots \times M_j) = min(Cost(M_i \times M_{i+1} \times \cdots \times M_k) + Cost(M_{k+1} \times \cdots \times M_j) + (r_i c_k c_j)$

## Final Equation

- Base case (No multiplication) : $Cost(i,i) = 0$ 
- $Cost(i,j) = min_{i \leq k < j} ([Cost(i,k) + Cost(k+1,j) + (r_i c_k c_j)])$ only when $i \leq j$

## Dynamic Programming Approach:

1. **Subproblem Definition:**

   * Let `C[i, j]` represent the minimum number of scalar multiplications needed to compute the product of matrices `A[i]` to `A[j]`.

2. **Base Case:**

   * `C[i, i] = 0` for all `i`.
3. **Recursive Relation:**

   * If `i == j`, `C[i, j] = 0` (single matrix, no multiplication needed).
   * If `i < j`:
     * `C[i, j] = min(C[i, k] + C[k+1, j] + p[i-1] * p[k] * p[j])` for `k = i` to `j-1`, where `p[i-1]` is the number of rows in `A[i]` and `p[k]` is the number of columns in `A[k]`.

4. **Algorithm:**
* The algorithm uses a 2D table `C` to store the minimum cost of multiplying subchains of matrices.
* It iterates through different chain lengths and calculates the minimum cost for each subchain.
* The final result `C[0][n-1]` gives the minimum cost for multiplying the entire chain of matrices.

This dynamic programming approach efficiently solves the matrix chain multiplication problem by breaking it down into smaller overlapping subproblems and avoiding redundant calculations.

In [3]:
def matrix_chain_order(p):
    n = len(p) - 1  # Number of matrices

    # Initialize C[i, j] for all i and j
    C = [[0 for _ in range(n)] for _ in range(n)]

    # Compute C[i, j] for all chain lengths
    for L in range(2, n + 1):  # L is chain length
        for i in range(1, n - L + 2):
            j = i + L - 1
            C[i - 1][j - 1] = float('inf')
            for k in range(i, j):
                q = C[i - 1][k - 1] + C[k][j - 1] + p[i - 1] * p[k] * p[j]
                if q < C[i - 1][j - 1]:
                    C[i - 1][j - 1] = q

    return C[0][n - 1]  # Minimum number of scalar multiplications

# Example usage:
p = [30, 35, 15, 5, 10, 20, 25]  # Dimensions of matrices 
min_multiplications = matrix_chain_order(p)
print("Minimum number of scalar multiplications:", min_multiplications)


Minimum number of scalar multiplications: 15125


## Time Complexity

* **Dynamic Programming:** $O(n^3)$ 
    * We have three nested loops:
        * Outer loop iterates through chain lengths (L).
        * Middle loop iterates through the starting index (i) of the subchain.
        * Inner loop iterates through possible split points (k) within the subchain.

* **Brute Force:** $O(2^n)$ 
    * In brute force, we would explore all possible parenthesizations, leading to an exponential number of combinations.

## Space Complexity:

* **Dynamic Programming:** $O(n^2)$
    * We use a 2D array `m` of size n x n to store the minimum cost for each subchain.

* **Brute Force:** $O(n)$ (approximately) 
    * Primarily uses recursion stack space, which can grow linearly with the number of matrices in the worst case.